In [1]:
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt

###############################################################################
# 1) PRICER BLACK-SCHOLES POUR CALLS ET PUTS
###############################################################################

def black_scholes_call(S, K, T, r, sigma):
    """
    Prix d'un call européen selon le modèle Black-Scholes.
    """
    if T <= 0:
        return max(S - K, 0)
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return S * st.norm.cdf(d1) - K * np.exp(-r*T) * st.norm.cdf(d2)

def black_scholes_put(S, K, T, r, sigma):
    """
    Prix d'un put européen selon le modèle Black-Scholes.
    """
    if T <= 0:
        return max(K - S, 0)
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return K * np.exp(-r*T) * st.norm.cdf(-d2) - S * st.norm.cdf(-d1)

In [2]:
###############################################################################
# 2) OUTIL DE VOLATILITÉ IMPLICITE
###############################################################################

def implied_vol_call_bisection(price_market, S, K, T, r,
                               sigma_lower=1e-6, sigma_upper=3.0,
                               tol=1e-7, max_iter=100):
    """
    Calcule la volatilité implicite (pour un call) via la méthode de bissection.
    price_market : prix observé de l'option sur le marché
    """
    # Vérification rapide
    if price_market < max(0, S - K*np.exp(-r*T)):
        return 0.0  # En principe, en deçà de l'intrinsèque => incohérence
    for _ in range(max_iter):
        sigma_mid = 0.5*(sigma_lower + sigma_upper)
        price_mid = black_scholes_call(S, K, T, r, sigma_mid)
        if abs(price_mid - price_market) < tol:
            return sigma_mid
        if price_mid < price_market:
            sigma_lower = sigma_mid
        else:
            sigma_upper = sigma_mid
    return sigma_mid

In [3]:
###############################################################################
# 3) VÉRIFIER L'ABSENCE D'ARBITRAGE
###############################################################################

def check_call_arbitrage(strikes, call_prices):
    """
    Vérifie grossièrement si la structure de prix (call_prices) est monotone
    décroissante en fonction des strikes (à même maturité).
    Renvoie True si pas d'arbitrage détecté, False sinon.
    """
    # Condition simple : C(K1) >= C(K2) si K1 < K2
    for i in range(len(strikes) - 1):
        if strikes[i] < strikes[i+1]:
            if call_prices[i] < call_prices[i+1]:
                # Arbitrage potentiel détecté
                return False
    return True

In [4]:
###############################################################################
# 4) PDE DE BLACK-SCHOLES (SCHÉMA D’EULER EXPLICITE)
###############################################################################

def bs_pde_euler_call(K, T, r, sigma, S_max, M, N):
    """
    Résout la PDE de Black-Scholes par un schéma explicite (Euler) pour un call.
    
    PDE: 
        ∂u/∂t + 0.5*sigma^2 * S^2 * ∂²u/∂S² + r*S*∂u/∂S - r*u = 0
    avec condition terminale u(T,S) = max(S-K,0) et conditions aux frontières:
        u(t,0) = 0,
        u(t, S_max) ~ S_max - K e^{-r(T-t)} (pour un call).
    
    Paramètres:
        K     : Strike
        T     : Maturité
        r     : Taux sans risque
        sigma : Volatilité (constante)
        S_max : Valeur max de S sur la grille
        M     : Nombre de points en espace
        N     : Nombre de points en temps
    Retourne:
        S_grid : grille des prix du sous-jacent
        u[0,:] : valeur de l'option à t=0 pour tous les S de la grille
    """
    # Discrétisation
    dS = S_max / M
    dt = T / N
    
    S_grid = np.linspace(0, S_max, M+1)
    
    # Matrice solution: u[n,j] ~ valeur à l'instant n (dans le temps) et position j (en S)
    u = np.zeros((N+1, M+1))
    
    # Condition terminale à t = T
    u[N, :] = np.maximum(S_grid - K, 0)
    
    # Conditions aux frontières
    # S=0 => payoff call = 0
    # S=S_max => approx: u(t, S_max) = S_max - K*exp(-r*(T-t))
    for n in range(N+1):
        t = n*dt
        u[n, 0] = 0.0
        u[n, M] = S_max - K*np.exp(-r*(T - t))
    
    # On recule dans le temps (du pas N-1 jusqu'à 0)
    for n in range(N-1, -1, -1):
        for j in range(1, M):
            # Différences finies
            # Approx des dérivées
            d2u_dS2 = (u[n+1, j+1] - 2*u[n+1, j] + u[n+1, j-1]) / (dS**2)
            du_dS   = (u[n+1, j+1] - u[n+1, j-1]) / (2*dS)
            
            # Terme S^2
            S_j = S_grid[j]
            
            # Mise à jour explicite
            u[n, j] = u[n+1, j] + dt * (
                0.5*sigma**2 * S_j**2 * d2u_dS2 + r*S_j*du_dS - r*u[n+1, j]
            )
    
    # u[0,:] est la valeur de l'option au temps t=0 pour tous les S
    return S_grid, u[0,:]

In [5]:
###############################################################################
# 5) - 6) - 7) - 8) DÉMO COMPLÈTE (MAIN)
###############################################################################

def main():
    """
    Démonstration complète:
    1) Pricing BS (calls & puts)
    2) Vol implicite
    3) Vérif arbitrage
    4) PDE Euler
    5) Pricing sur plusieurs strikes (85, 90, 95, 100, 105, 110)
    6) Variation des paramètres
    7) Implied vol à partir des prix PDE
    8) Commentaires
    """
    print("=== 1) Pricer Black-Scholes (Call & Put) ===")
    S0 = 100.0
    K_call = 100.0
    T = 1.0
    r = 0.05
    sigma = 0.20
    
    call_price_bs = black_scholes_call(S0, K_call, T, r, sigma)
    put_price_bs  = black_scholes_put(S0, K_call, T, r, sigma)
    print(f"Call BS (S0=100, K=100): {call_price_bs:.4f}")
    print(f"Put  BS (S0=100, K=100): {put_price_bs:.4f}\n")
    
    print("=== 2) Outil de volatilité implicite (à partir du prix BS) ===")
    # On récupère la vol implicite en partant du prix BS
    vol_imp = implied_vol_call_bisection(call_price_bs, S0, K_call, T, r)
    print(f"Volatilité implicite retrouvée : {vol_imp:.4f}\n")
    
    print("=== 3) Vérifier l'absence d'arbitrage sur plusieurs strikes ===")
    strikes = [85, 90, 95, 100, 105, 110]
    call_prices_bs = [black_scholes_call(S0, K, T, r, sigma) for K in strikes]
    no_arbitrage = check_call_arbitrage(strikes, call_prices_bs)
    print("Strikes :", strikes)
    print("Call prices BS :", [f"{cp:.2f}" for cp in call_prices_bs])
    print("Pas d'arbitrage détecté ?" , no_arbitrage, "\n")
    
    print("=== 4) Résolution PDE Black-Scholes (schéma d'Euler explicite) ===")
    S_max = 3*S0  # On va jusqu'à 300 pour le sous-jacent
    M = 200       # Points en espace
    N = 200       # Points en temps
    
    # Prix PDE pour K=100
    S_grid, pde_solution = bs_pde_euler_call(K_call, T, r, sigma, S_max, M, N)
    # On cherche l'indice le plus proche de S0=100
    idx_100 = np.argmin(np.abs(S_grid - S0))
    call_price_pde_100 = pde_solution[idx_100]
    print(f"Call PDE (S0=100, K=100) : {call_price_pde_100:.4f}")
    print(f"Comparaison vs Formule BS : {call_price_bs:.4f}\n")
    
    print("=== 5) Prix PDE sur plusieurs strikes (85, 90, 95, 100, 105, 110) ===")
    pde_prices_multi = []
    for K_ in strikes:
        _, pde_sol_ = bs_pde_euler_call(K_, T, r, sigma, S_max, M, N)
        idx_ = np.argmin(np.abs(S_grid - S0))
        pde_prices_multi.append(pde_sol_[idx_])
    
    for K_, p_ in zip(strikes, pde_prices_multi):
        print(f"Strike={K_}, Price PDE={p_:.4f}, Price BS={black_scholes_call(S0, K_, T, r, sigma):.4f}")
    print()
    
    print("=== 6) Variation des paramètres ===")
    # On fait varier la volatilité par exemple
    sigmas_test = [0.10, 0.20, 0.30]
    for sig_ in sigmas_test:
        c_bs = black_scholes_call(S0, K_call, T, r, sig_)
        print(f"Vol={sig_}, Call BS(K=100)={c_bs:.4f}")
    print()
    
    print("=== 7) Calcul de la vol implicite à partir des prix PDE ===")
    # On calcule la vol implicite pour chaque strike (avec le prix PDE)
    implied_vols_from_pde = []
    for K_, p_ in zip(strikes, pde_prices_multi):
        vol_pde = implied_vol_call_bisection(p_, S0, K_, T, r)
        implied_vols_from_pde.append(vol_pde)
    
    print("Strike | PDE Price | Implied Vol")
    for K_, pp_, iv_ in zip(strikes, pde_prices_multi, implied_vols_from_pde):
        print(f"{K_:6.1f} | {pp_:9.4f} | {iv_:11.4f}")
    print()
    
    
    print("=== 8) Commentaires ===")
    print("a) The Black-Scholes method (for calls and puts) and the implied volatility tool work correctly, as confirmed by the BS prices obtained (Call = 10.4506, Put = 5.5735).")
    print("b) The implied volatility found from the BS prices is 0.2000, demonstrating the consistency of our implementation.")
    print("c) The arbitrage check across multiple strikes shows that call prices decrease (and put prices increase) with strike, ensuring no arbitrage opportunities.")
    print("d) The PDE solution using an explicit Euler scheme exhibits numerical instability: for S0=100 and K=100, the PDE price is aberrant (negative and enormous) while the BS price is 10.4506.")
    print("e) For several strikes, the PDE prices diverge and remain negative, confirming that the explicit scheme used is not stable with the chosen grid.")
    print("f) The variation in volatility shows that the call (and put) price increases with volatility, which is in line with theoretical expectations (for σ = 0.10, 0.20, 0.30, the prices evolve as expected).")
    print("g) The implied volatility computed from the PDE prices returns 0.0000 for all strikes, because the erroneous (negative) PDE prices cannot be reproduced by the BS formula.")
    print("h) In conclusion, our BS pricer and implied volatility tool work correctly, while the numerical method used for the PDE requires refinement (a finer grid or an implicit/Crank-Nicolson scheme) to obtain consistent results.")


# Lancement de la démonstration
if __name__ == "__main__":
    main()

=== 1) Pricer Black-Scholes (Call & Put) ===
Call BS (S0=100, K=100): 10.4506
Put  BS (S0=100, K=100): 5.5735

=== 2) Outil de volatilité implicite (à partir du prix BS) ===
Volatilité implicite retrouvée : 0.2000

=== 3) Vérifier l'absence d'arbitrage sur plusieurs strikes ===
Strikes : [85, 90, 95, 100, 105, 110]
Call prices BS : ['20.47', '16.70', '13.35', '10.45', '8.02', '6.04']
Pas d'arbitrage détecté ? True 

=== 4) Résolution PDE Black-Scholes (schéma d'Euler explicite) ===
Call PDE (S0=100, K=100) : -16393739817005174980713922495671870052988368024236731154977336059223615721804671726858603286416236670620183988011008.0000
Comparaison vs Formule BS : 10.4506

=== 5) Prix PDE sur plusieurs strikes (85, 90, 95, 100, 105, 110) ===
Strike=85, Price PDE=-13934678872889861799859837229625600685544773397229777932480317858678856348428067354557159211930022370628171888328704.0000, Price BS=20.4693
Strike=90, Price PDE=-147543658994158200894249726215667924519027117716136862498537063197504314